In [1]:
import pandas
import requests
import spotipy
import string
import re

from spotipy.oauth2 import SpotifyClientCredentials
from genderize import Genderize

"""
# Not really working
# From 'Song for Bob Dylan', extracted the name 'Song' :(
import spacy
import spacy.cli
spacy.cli.download('en_core_web_lg')
en = spacy.load('en_core_web_lg')

"""


"\n# Not really working\n# From 'Song for Bob Dylan', extracted the name 'Song' :(\nimport spacy\nimport spacy.cli\nspacy.cli.download('en_core_web_lg')\nen = spacy.load('en_core_web_lg')\n\n"

In [4]:
with open('vars.env', 'r') as f:
    my_id = f.readline().rstrip('\n')
    my_secret = f.readline().rstrip('\n')
    playlistURI = f.readline().rstrip('\n')
    owner = f.readline().rstrip('\n')

client_credentials_manager = SpotifyClientCredentials(client_id=my_id, client_secret=my_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlistID = playlistURI[-22:]

In [5]:
results = sp.user_playlist(owner, playlist_id=playlistID, fields='tracks, next')['tracks']

songs =  results['items']
while results['next']:
    results = sp.next(results)
    songs.extend(results['items'])

idSet = {track['added_by']['id'] for track in songs}    
us = {(id, sp.user(id)['display_name']) for id in idSet}
us = dict(us)
us['rmb101'] = 'rmb101'

print(len(songs))


241


In [14]:
# Make the dataframe by pulling out the song name, artist and person who added it
table = [(track['track']['name'], track['track']['artists'][0]['name'], us[track['added_by']['id']]) 
         for track in songs]
df = pandas.DataFrame(table, columns = ['Song', 'Artist', 'Added by'])

# Get rid of anything after a hyphen, as this is all '2012 remastered' kinda text
df['Song'].replace(' - .*','', regex = True, inplace = True)

# set up regex to kill off punctuation 
punc = re.compile('[%s]' % re.escape(string.punctuation))

# bring in list of common words (ie not names) to remove
# if this works I might expand it
words = pandas.read_csv('commonwords.csv')
wordlist = words['Edited top 1000 words from www.wordfrequency.info'].tolist()

# function to remove the punctuation and common names
# returns as list because I'm going to run genderize over it as a list
def namesOnly(row):
    text = punc.sub('', row['Song'].lower()).split()
    newTitle = [word for word in text if word not in wordlist]
    return newTitle

df['ShortTitle'] = df.apply(namesOnly, axis = 1)


In [16]:
# Function to try to assign a gender to each word in each shortened song title
def gender(song):
    response = Genderize().get(song['ShortTitle'])
    return response
    #return [(word['gender'], word['probability']) for word in response] 

df['GenderData'] = df.apply(gender, axis = 1)

df

,Song,Artist,Added by,ShortTitle,GenderData
0,Billie Jean,Michael Jackson,Diana Be,"[billie, jean]","[{'name': 'billie', 'gender': 'female', 'proba..."
1,Ziggy Stardust,David Bowie,Tom Williams,"[ziggy, stardust]","[{'name': 'ziggy', 'gender': 'male', 'probabil..."
2,Dirty Diana,Michael Jackson,Diana Be,"[dirty, diana]","[{'name': 'dirty', 'gender': 'male', 'probabil..."
3,Johnny Too Bad,John Martyn,rjdw500,[johnny],"[{'name': 'johnny', 'gender': 'male', 'probabi..."
4,Jolene,Dolly Parton,danielnissenbaum,[jolene],"[{'name': 'jolene', 'gender': 'female', 'proba..."
5,Roxanne,The Police,Diana Be,[roxanne],"[{'name': 'roxanne', 'gender': 'female', 'prob..."
6,Letter To Hermione,David Bowie,Chelle Cole,[hermione],"[{'name': 'hermione', 'gender': 'female', 'pro..."
7,Rhiannon,Fleetwood Mac,rjdw500,[rhiannon],"[{'name': 'rhiannon', 'gender': 'female', 'pro..."
8,Janine,David Bowie,Chelle Cole,[janine],"[{'name': 'janine', 'gender': 'female', 'proba..."
9,Maria,Blondie,Greg,[maria],"[{'name': 'maria', 'gender': 'female', 'probab..."


In [27]:
# Make this part of main genderising function once happy with it
def tidy(song):
    words = [[word['gender'], word['probability']] for word in song['GenderData'] if word['gender']]
    guesses = [word[0] for word in words if word[1] > 0.7] 
    if len(set(guesses)) == 1:
        return guesses[0]
    elif not guesses:
        return ''
    else:
        return guesses

df['Guesses'] = df.apply(tidy, axis = 1)

df

,Song,Artist,Added by,ShortTitle,GenderData,Guesses
0,Billie Jean,Michael Jackson,Diana Be,"[billie, jean]","[{'name': 'billie', 'gender': 'female', 'proba...",female
1,Ziggy Stardust,David Bowie,Tom Williams,"[ziggy, stardust]","[{'name': 'ziggy', 'gender': 'male', 'probabil...",male
2,Dirty Diana,Michael Jackson,Diana Be,"[dirty, diana]","[{'name': 'dirty', 'gender': 'male', 'probabil...","[male, female]"
3,Johnny Too Bad,John Martyn,rjdw500,[johnny],"[{'name': 'johnny', 'gender': 'male', 'probabi...",male
4,Jolene,Dolly Parton,danielnissenbaum,[jolene],"[{'name': 'jolene', 'gender': 'female', 'proba...",female
5,Roxanne,The Police,Diana Be,[roxanne],"[{'name': 'roxanne', 'gender': 'female', 'prob...",female
6,Letter To Hermione,David Bowie,Chelle Cole,[hermione],"[{'name': 'hermione', 'gender': 'female', 'pro...",female
7,Rhiannon,Fleetwood Mac,rjdw500,[rhiannon],"[{'name': 'rhiannon', 'gender': 'female', 'pro...",female
8,Janine,David Bowie,Chelle Cole,[janine],"[{'name': 'janine', 'gender': 'female', 'proba...",female
9,Maria,Blondie,Greg,[maria],"[{'name': 'maria', 'gender': 'female', 'probab...",female


# Not really working

def entity(text):
    nameList = [name for name in en(text['Song']).ents if name.label_ == 'PERSON']
    if nameList:
        return nameList[0]
    else:
        return None

df['SongNames'] = df.apply(entity, axis = 1)
df.head(50)